# Final Project

Team name: Stack Overflow

Members:
Ki Jeong,
Moneeb Abu-Esba,
Omar Diaz,
Luke Lopez

# Project Overview

Per annotation guidelines, our project dataset(Gold_Annotations.xlsx) had 3 columns: Comment, Tech, and Subtopic.

Comment: This contained the user's reddit post.
Tech: This contained either "tech" or "nontech" depending on the post.
Subtopic: This contained "education", "mobility", "recreation", or "other" depending on the post.

We decided to build models for each Tech and Subtopic categories.


# Inter-annotator agreement

First, we will calculate the agreement scores between the annotators for "Tech" and "Subtopic". We will also include Cohen's Kappa to compare the annotator's accuracy against the true labels.



In [60]:
import pandas as pd
import re
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import cohen_kappa_score

InteractiveShell.ast_node_interactivity = 'all'

file_path_1 = 'Stack Overflow 1000 Samples (Reddit Miners).xlsx'
df_1 = pd.read_excel(file_path_1)

file_path_2 = 'Stack Overflow 1000 Samples (Data Scavenger).xlsx'
df_2 = pd.read_excel(file_path_2)

if 'Tech' not in df_1.columns or 'Tech' not in df_2.columns:
    raise ValueError("The 'Tech' column is missing in one or both files.")

if 'Subtopic' not in df_1.columns or 'Subtopic' not in df_2.columns:
    raise ValueError("The 'Subtopic' column is missing in one or both files.")

tech_matches = (df_1['Tech'] == df_2['Tech']).astype(int).values
tech_one_count = sum(tech_matches)
#print("Matching (1) and Non-matching (0) in 'Tech' column:")
#print(tech_matches)

subtopic_matches = (df_1['Subtopic'] == df_2['Subtopic']).astype(int).values
subtopic_one_count = sum(subtopic_matches)
#print("\nMatching (1) and Non-matching (0) in 'Subtopic' column:"
#print(subtopic_matches)


print("Between Reddit Miners and Data Scavengers, there were {:.0f} out of 1000 Tech annotation agreements." .format(tech_one_count))
print("Between Reddit Miners and Data Scavengers, there were {:.0f} out of 1000 Subtopic annotation agreements." .format(subtopic_one_count))
print("\n")

data = pd.read_excel('Gold_Annotations.xlsx')
data_tech = data['Tech']
data_subtopic = data['Subtopic']

df_1_tech = df_1['Tech']
df_2_tech = df_2['Tech']
df_1_subtopic = df_1['Subtopic']
df_2_subtopic = df_2['Subtopic']

true_tech_labels = []
tmp = ''
for feature in data_tech:
    tmp = feature
    true_tech_labels.append(tmp)
    
true_subtopic_labels = []
tmp = ''
for feature in data_subtopic:
    tmp = feature
    true_subtopic_labels.append(tmp)
    
df_1_tech_labels = []
tmp = ''
for feature in df_1_tech:
    tmp = feature
    df_1_tech_labels.append(tmp)

df_2_tech_labels = []
tmp = ''
for feature in df_2_tech:
    tmp = feature
    df_2_tech_labels.append(tmp)
    
df_1_subtopic_labels = []
tmp = ''
for feature in df_1_subtopic:
    tmp = feature
    df_1_subtopic_labels.append(tmp)

df_2_subtopic_labels = []
tmp = ''
for feature in df_2_subtopic:
    tmp = feature
    df_2_subtopic_labels.append(tmp)

rm_tech_kappa = cohen_kappa_score(true_tech_labels, df_1_tech_labels)
ds_tech_kappa = cohen_kappa_score(true_tech_labels, df_2_tech_labels)

rm_subtopic_kappa = cohen_kappa_score(true_subtopic_labels, df_1_subtopic_labels)
ds_subtopic_kappa = cohen_kappa_score(true_subtopic_labels, df_2_subtopic_labels)
print("Reddit Miner Tech Accuracy: Cohen's kappa:", rm_tech_kappa)
print("Data Scavenger Tech Accuracy: Cohen's kappa:", ds_tech_kappa)
print("\n")
print("Reddit Miner Subtopic Accuracy: Cohen's kappa:", rm_subtopic_kappa)
print("Data Scavenger Subtopic Accuracy: Cohen's kappa:", ds_subtopic_kappa)

Between Reddit Miners and Data Scavengers, there were 542 out of 1000 Tech annotation agreements.
Between Reddit Miners and Data Scavengers, there were 392 out of 1000 Subtopic annotation agreements.


Reddit Miner Tech Accuracy: Cohen's kappa: 0.46841239478123287
Data Scavenger Tech Accuracy: Cohen's kappa: 0.5958644772722814


Reddit Miner Subtopic Accuracy: Cohen's kappa: 0.30246100476755455
Data Scavenger Subtopic Accuracy: Cohen's kappa: 0.3923983416048853


# Tech Variable

Next, we will address the Tech variable. We are using cross-validation (cv=5) where applicable.



# Lexicon Feature

Simple lexicon consisting of 9 tech words are added and tested on the Tech variable.

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from collections import Counter

data = pd.read_excel('Gold_Annotations.xlsx')
X = data['Comment']
y = data['Tech']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=25)

class LexiconClassifier():
    def __init__(self):
        self.tech_words = ["computer", "phone", "app", "internet", "fiber", "google", "facebook", "waze", "tech"]
        
    def tech_predict(self, sentence):
        num_tech_words = 0
        for word in sentence.lower().split():
            if word in self.tech_words:
                num_tech_words += 1
        
        tech_pred = 'nontech'        
        if num_tech_words > 0:
            tech_pred = 'tech'
        return tech_pred

lex_class = LexiconClassifier()
tech_preds = []
tmp = ''
for feature in X_train:
    tmp = lex_class.tech_predict(feature)
    tech_preds.append(tmp)

precision_mac = precision_score(y_train, tech_preds, average='macro')
recall_mac = recall_score(y_train, tech_preds, average='macro')
f1_mac = f1_score(y_train, tech_preds, average='macro')
precision_min = precision_score(y_train, tech_preds, average='micro')
recall_min = recall_score(y_train, tech_preds, average='micro')
f1_min = f1_score(y_train, tech_preds, average='micro')
print("Validation Tech Lexicon Macro Precision: {:.4f}".format(precision_mac))
print("Validation Tech Lexicon Macro Recall: {:.4f}".format(recall_mac))
print("Validation Tech Lexicon Macro F1: {:.4f}".format(f1_mac))
print("Validation Tech Lexicon Micro Precision: {:.4f}".format(precision_min))
print("Validation Tech Lexicon Micro Recall: {:.4f}".format(recall_min))
print("Validation Tech Lexicon Micro F1: {:.4f}".format(f1_min))
print("\n")


tech_preds = []
tmp = ''
for feature in X_test:
    tmp = lex_class.tech_predict(feature)
    tech_preds.append(tmp)

precision_mac = precision_score(y_test, tech_preds, average='macro')
recall_mac = recall_score(y_test, tech_preds, average='macro')
f1_mac = f1_score(y_test, tech_preds, average='macro')
precision_min = precision_score(y_test, tech_preds, average='micro')
recall_min = recall_score(y_test, tech_preds, average='micro')
f1_min = f1_score(y_test, tech_preds, average='micro')
print("Test Tech Lexicon Macro Precision: {:.4f}".format(precision_mac))
print("Test Tech Lexicon Macro Recall: {:.4f}".format(recall_mac))
print("Test Tech Lexicon Macro F1: {:.4f}".format(f1_mac))
print("Test Tech Lexicon Micro Precision: {:.4f}".format(precision_min))
print("Test Tech Lexicon Micro Recall: {:.4f}".format(recall_min))
print("Test Tech Lexicon Micro F1: {:.4f}".format(f1_min))
print("\n")

tech_lexicon_kappa = cohen_kappa_score(y_test, tech_preds)
print("Tech Lexicon's Cohen's kappa:", tech_lexicon_kappa)

Validation Tech Lexicon Macro Precision: 0.6001
Validation Tech Lexicon Macro Recall: 0.5988
Validation Tech Lexicon Macro F1: 0.5994
Validation Tech Lexicon Micro Precision: 0.6240
Validation Tech Lexicon Micro Recall: 0.6240
Validation Tech Lexicon Micro F1: 0.6240


Test Tech Lexicon Macro Precision: 0.6418
Test Tech Lexicon Macro Recall: 0.6488
Test Tech Lexicon Macro F1: 0.6441
Test Tech Lexicon Micro Precision: 0.6720
Test Tech Lexicon Micro Recall: 0.6720
Test Tech Lexicon Micro F1: 0.6720


Tech Lexicon's Cohen's kappa: 0.28942807625649913


# Bag of Words(Unigram) Feature

This time, simple linear model using Bag of Words(unigram) feature is tested on the Tech variable.

In [65]:
import csv
import numpy as np
np.random.seed(42)
import random
random.seed(42)

vec = CountVectorizer(ngram_range=(1,1))
X_train_features = vec.fit_transform(X_train) # This should be a matrix
X_test_features = vec.transform(X_test) # This should be a matrix

model = LinearSVC(dual='auto')
params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

clf_f1_mac = GridSearchCV(model, params, scoring='f1_macro', cv=5)
clf_f1_mic = GridSearchCV(model, params, scoring='f1_micro', cv=5)
clf_prec_mac = GridSearchCV(model, params, scoring='precision_macro', cv=5)
clf_prec_mic = GridSearchCV(model, params, scoring='precision_micro', cv=5)
clf_recall_mac = GridSearchCV(model, params, scoring='recall_macro', cv=5)
clf_recall_mic = GridSearchCV(model, params, scoring='recall_micro', cv=5)

clf_f1_mac.fit(X_train_features, y_train)
clf_f1_mic.fit(X_train_features, y_train)
clf_prec_mac.fit(X_train_features, y_train)
clf_prec_mic.fit(X_train_features, y_train)
clf_recall_mac.fit(X_train_features, y_train)
clf_recall_mic.fit(X_train_features, y_train)

y_val_f1_mac = clf_f1_mac.best_score_
y_val_f1_mic = clf_f1_mic.best_score_
y_val_prec_mac = clf_prec_mac.best_score_
y_val_prec_mic = clf_prec_mic.best_score_
y_val_recall_mac = clf_recall_mac.best_score_
y_val_recall_mic = clf_recall_mic.best_score_

print("Validation Tech Bag of Words Macro Precision:", y_val_prec_mac)
print("Validation Tech Bag of Words Macro Recall:", y_val_recall_mac)
print("Validation Tech Bag of Words Macro F1 Score:", y_val_f1_mac)
print("Validation Tech Bag of Words Micro Precision:", y_val_prec_mic)
print("Validation Tech Bag of Words Micro Recall:", y_val_recall_mic)
print("Validation Tech Bag of Words Micro F1 Score:", y_val_f1_mic)
print("\n")



y_test_f1_mac = clf_f1_mac.predict(X_test_features)
y_test_f1_mic = clf_f1_mic.predict(X_test_features)
y_test_prec_mac = clf_prec_mac.predict(X_test_features)
y_test_prec_mic = clf_prec_mic.predict(X_test_features)
y_test_recall_mac = clf_recall_mac.predict(X_test_features)
y_test_recall_mic = clf_recall_mic.predict(X_test_features)

precision_macro = precision_score(y_test, y_test_prec_mac, average='macro')
recall_macro = recall_score(y_test, y_test_recall_mac, average='macro')
f1_macro = f1_score(y_test, y_test_f1_mac, average='macro')
precision_micro = precision_score(y_test, y_test_prec_mic, average='micro')
recall_micro = recall_score(y_test, y_test_recall_mic, average='micro')
f1_micro = f1_score(y_test, y_test_f1_mic, average='micro')

print("Test Tech Bag of Words Macro Precision:", precision_macro)
print("Test Tech Bag of Words Macro Recall:", recall_macro)
print("Test Tech Bag of Words Macro F1 Score:", f1_macro)
print("Test Tech Bag of Words Micro Precision:", precision_micro)
print("Test Tech Bag of Words Micro Recall:", recall_micro)
print("Test Tech Bag of Words Micro F1 Score:", f1_micro)
print("\n")

tech_bow_kappa = cohen_kappa_score(y_test, y_test_f1_mac)
print("Tech Bag of Words: Cohen's kappa:", tech_bow_kappa)

#my_file = open('tech_bestmodel.csv', 'w')    #code to output best model
#for label in y_test_prec_mac:
#   my_file.write(label + '\n')
#my_file.close()

#my_file = open('tech_testlabels.csv', 'w')    #code to output test labels
#for label in y_test:
#   my_file.write(label + '\n')
#my_file.close()

#path = 'testset.csv'        #code to output test set
#X_test.to_csv(path, index=False)

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: Conv

GridSearchCV(cv=5, estimator=LinearSVC(dual='auto'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='f1_macro')

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: Conv

GridSearchCV(cv=5, estimator=LinearSVC(dual='auto'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='f1_micro')

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: Conv

...

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: Conv

GridSearchCV(cv=5, estimator=LinearSVC(dual='auto'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='recall_macro')

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: Conv

GridSearchCV(cv=5, estimator=LinearSVC(dual='auto'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='recall_micro')

Validation Tech Bag of Words Macro Precision: 0.6452905895985518
Validation Tech Bag of Words Macro Recall: 0.5663054485662963
Validation Tech Bag of Words Macro F1 Score: 0.5648668159844746
Validation Tech Bag of Words Micro Precision: 0.632
Validation Tech Bag of Words Micro Recall: 0.632
Validation Tech Bag of Words Micro F1 Score: 0.632


Test Tech Bag of Words Macro Precision: 0.6683848797250859
Test Tech Bag of Words Macro Recall: 0.6703208556149733
Test Tech Bag of Words Macro F1 Score: 0.6577341581310185
Test Tech Bag of Words Micro Precision: 0.69
Test Tech Bag of Words Micro Recall: 0.69
Test Tech Bag of Words Micro F1 Score: 0.69


Tech Bag of Words: Cohen's kappa: 0.32286473911598546


# Weblink Feature

Next, weblink presence is checked in comments. If present, Tech label is set to "nontech".

In [47]:
import re

class WeblinkClassifier():
    def __init__(self):
        self.http_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        
    def tech_web_predict(self, sentence):
        weblink_count = 0
        for word in sentence.lower().split():
            weblink_count = len(re.findall(self.http_pattern, sentence))
        tech_web_pred = 'tech'
        if weblink_count > 0:
            tech_web_pred = 'nontech'
        return tech_web_pred

web_class = WeblinkClassifier()
tech_web_preds = []
tmp = ''
for feature in X_train:
    tmp = web_class.tech_web_predict(feature)
    tech_web_preds.append(tmp)

precision_mac = precision_score(y_train, tech_web_preds, average='macro')
recall_mac = recall_score(y_train, tech_web_preds, average='macro')
f1_mac = f1_score(y_train, tech_web_preds, average='macro')
precision_min = precision_score(y_train, tech_web_preds, average='micro')
recall_min = recall_score(y_train, tech_web_preds, average='micro')
f1_min = f1_score(y_train, tech_web_preds, average='micro')

print("Validation Tech Weblink Macro Precision: {:.4f}".format(precision_mac))
print("Validation Tech Weblink Macro Recall: {:.4f}".format(recall_mac))
print("Validation Tech Weblink Macro F1: {:.4f}".format(f1_mac))
print("Validation Tech Weblink Micro Precision: {:.4f}".format(precision_min))
print("Validation Tech Weblink Micro Recall: {:.4f}".format(recall_min))
print("Validation Tech Weblink Micro F1: {:.4f}".format(f1_min))
print("\n")



tech_web_preds = []
tmp = ''
for feature in X_test:
    tmp = web_class.tech_web_predict(feature)
    tech_web_preds.append(tmp)

precision_mac = precision_score(y_test, tech_web_preds, average='macro')
recall_mac = recall_score(y_test, tech_web_preds, average='macro')
f1_mac = f1_score(y_test, tech_web_preds, average='macro')
precision_min = precision_score(y_test, tech_web_preds, average='micro')
recall_min = recall_score(y_test, tech_web_preds, average='micro')
f1_min = f1_score(y_test, tech_web_preds, average='micro')
print("Test Tech Weblink Macro Precision: {:.4f}".format(precision_mac))
print("Test Tech Weblink Macro Recall: {:.4f}".format(recall_mac))
print("Test Tech Weblink Macro F1: {:.4f}".format(f1_mac))
print("Test Tech Weblink Micro Precision: {:.4f}".format(precision_min))
print("Test Tech Weblink Micro Recall: {:.4f}".format(recall_min))
print("Test Tech Weblink Micro F1: {:.4f}".format(f1_min))
print("\n")

tech_web_kappa = cohen_kappa_score(y_test, tech_web_preds)
print("Tech Weblink's Cohen's kappa:", tech_web_kappa)

Validation Tech Weblink Macro Precision: 0.5756
Validation Tech Weblink Macro Recall: 0.5550
Validation Tech Weblink Macro F1: 0.5461
Validation Tech Weblink Micro Precision: 0.6180
Validation Tech Weblink Micro Recall: 0.6180
Validation Tech Weblink Micro F1: 0.6180


Test Tech Weblink Macro Precision: 0.6280
Test Tech Weblink Macro Recall: 0.5849
Test Tech Weblink Macro F1: 0.5826
Test Tech Weblink Micro Precision: 0.6780
Test Tech Weblink Micro Recall: 0.6780
Test Tech Weblink Micro F1: 0.6780


Tech Weblink's Cohen's kappa: 0.19144234632382484


# Subtopic Variable

Next, the Subtopic variable is addressed. The values can be one of four: education, mobility, recreation, or other.

# Lexicon Feature

Simple lexicon consisting of 5 related words are added for 3 of 4 subtopic categories: education, mobility, recreation. The final topic, other, is defined as the default category.

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from collections import Counter

data = pd.read_excel('Gold_Annotations.xlsx')
X = data['Comment']
y = data['Subtopic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=25)

class LexiconClassifier():
    def __init__(self):
        self.education_words = ["school", "utsa", "learn", "degree", "education"]
        self.mobility_words = ["road", "drive", "car", "traffic", "map"]
        self.recreation_words = ["park", "play", "club", "drink", "restaurant"]
        
    def subtopic_predict(self, sentence):
        num_education_words = 0
        num_mobility_words = 0     
        num_recreation_words = 0     
        for word in sentence.lower().split():
            if word in self.education_words:
                num_education_words += 1
            if word in self.mobility_words:
                num_mobility_words += 1
            if word in self.recreation_words:
                num_recreation_words += 1
        
        subtopic_pred = 'other'        
        if num_education_words > num_mobility_words and num_education_words > num_recreation_words:
            subtopic_pred = 'education'
        elif num_mobility_words > num_education_words and num_mobility_words > num_recreation_words:
            subtopic_pred = 'mobility'
        elif num_recreation_words > num_education_words and num_recreation_words > num_mobility_words:
            subtopic_pred = 'recreation'

        return subtopic_pred

lex_class = LexiconClassifier()
subtopic_preds = []
tmp = ''
for feature in X_train:
    tmp = lex_class.subtopic_predict(feature)
    subtopic_preds.append(tmp)

precision_mac = precision_score(y_train, subtopic_preds, average='macro', zero_division=0)
recall_mac = recall_score(y_train, subtopic_preds, average='macro')
f1_mac = f1_score(y_train, subtopic_preds, average='macro')
precision_min = precision_score(y_train, subtopic_preds, average='micro', zero_division=0)
recall_min = recall_score(y_train, subtopic_preds, average='micro')
f1_min = f1_score(y_train, subtopic_preds, average='micro')
print("Validation Subtopic Lexicon Macro Precision: {:.4f}".format(precision_mac))
print("Validation Subtopic Lexicon Macro Recall: {:.4f}".format(recall_mac))
print("Validation Subtopic Lexicon Macro F1: {:.4f}".format(f1_mac))
print("Validation Subtopic Lexicon Micro Precision: {:.4f}".format(precision_min))
print("Validation Subtopic Lexicon Micro Recall: {:.4f}".format(recall_min))
print("Validation Subtopic Lexicon Micro F1: {:.4f}".format(f1_min))
print("\n")


subtopic_preds = []
tmp = ''
for feature in X_test:
    tmp = lex_class.subtopic_predict(feature)
    subtopic_preds.append(tmp)

precision_mac = precision_score(y_test, subtopic_preds, average='macro', zero_division=0)
recall_mac = recall_score(y_test, subtopic_preds, average='macro')
f1_mac = f1_score(y_test, subtopic_preds, average='macro')
precision_min = precision_score(y_test, subtopic_preds, average='micro', zero_division=0)
recall_min = recall_score(y_test, subtopic_preds, average='micro')
f1_min = f1_score(y_test, subtopic_preds, average='micro')
print("Test Subtopic Lexicon Macro Precision: {:.4f}".format(precision_mac))
print("Test Subtopic Lexicon Macro Recall: {:.4f}".format(recall_mac))
print("Test Subtopic Lexicon Macro F1: {:.4f}".format(f1_mac))
print("Test Subtopic Lexicon Micro Precision: {:.4f}".format(precision_min))
print("Test Subtopic Lexicon Micro Recall: {:.4f}".format(recall_min))
print("Test Subtopic Lexicon Micro F1: {:.4f}".format(f1_min))
print("\n")

subtopic_bow_kappa = cohen_kappa_score(y_test, subtopic_preds)
print("Subtopic Lexicon's Cohen's kappa:", subtopic_bow_kappa)

#my_file = open('subtopic_bestmodel.csv', 'w')    #code to output best model
#for label in subtopic_preds:
#   my_file.write(label + '\n')
#my_file.close()

#my_file = open('subtopic_testlabels.csv', 'w')    #code to output test labels
#for label in y_test:
#   my_file.write(label + '\n')
#my_file.close()

Validation Subtopic Lexicon Macro Precision: 0.4590
Validation Subtopic Lexicon Macro Recall: 0.3545
Validation Subtopic Lexicon Macro F1: 0.3887
Validation Subtopic Lexicon Micro Precision: 0.7700
Validation Subtopic Lexicon Micro Recall: 0.7700
Validation Subtopic Lexicon Micro F1: 0.7700


Test Subtopic Lexicon Macro Precision: 0.5757
Test Subtopic Lexicon Macro Recall: 0.4887
Test Subtopic Lexicon Macro F1: 0.5074
Test Subtopic Lexicon Micro Precision: 0.7660
Test Subtopic Lexicon Micro Recall: 0.7660
Test Subtopic Lexicon Micro F1: 0.7660


Subtopic Lexicon's Cohen's kappa: 0.30880476393024237


# Bag of Words(Unigram) Feature

This time, simple linear model using Bag of Words(unigram) feature is tested on the Subtopic variable.

In [31]:
vec = CountVectorizer(ngram_range=(1,1))
X_train_features = vec.fit_transform(X_train) # This should be a matrix
X_test_features = vec.transform(X_test) # This should be a matrix

model = LinearSVC(dual='auto')
params = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

f1_mac = GridSearchCV(model, params, scoring='f1_macro', cv=5)
f1_mic = GridSearchCV(model, params, scoring='f1_micro', cv=5)
prec_mac = GridSearchCV(model, params, scoring='precision_macro', cv=5)
prec_mic = GridSearchCV(model, params, scoring='precision_micro', cv=5)
recall_mac = GridSearchCV(model, params, scoring='recall_macro', cv=5)
recall_mic = GridSearchCV(model, params, scoring='recall_micro', cv=5)

f1_mac.fit(X_train_features, y_train)
f1_mic.fit(X_train_features, y_train)
prec_mac.fit(X_train_features, y_train)
prec_mic.fit(X_train_features, y_train)
recall_mac.fit(X_train_features, y_train)
recall_mic.fit(X_train_features, y_train)

y_val_f1_mac = f1_mac.best_score_
y_val_f1_mic = f1_mic.best_score_
y_val_prec_mac = prec_mac.best_score_
y_val_prec_mic = prec_mic.best_score_
y_val_recall_mac = recall_mac.best_score_
y_val_recall_mic = recall_mic.best_score_

print("Validation Subtopic Bag of Words Macro Precision:", y_val_prec_mac)
print("Validation Subtopic Bag of Words Macro Recall:", y_val_recall_mac)
print("Validation Subtopic Bag of Words Macro F1 Score:", y_val_f1_mac)
print("Validation Subtopic Bag of Words Micro Precision:", y_val_prec_mic)
print("Validation Subtopic Bag of Words Micro Recall:", y_val_recall_mic)
print("Validation Subtopic Bag of Words Micro F1 Score:", y_val_f1_mic)
print("\n")



y_test_f1_mac = f1_mac.predict(X_test_features)
y_test_f1_mic = f1_mic.predict(X_test_features)
y_test_prec_mac = prec_mac.predict(X_test_features)
y_test_prec_mic = prec_mic.predict(X_test_features)
y_test_recall_mac = recall_mac.predict(X_test_features)
y_test_recall_mic = recall_mic.predict(X_test_features)

precision_macro = precision_score(y_test, y_test_prec_mac, average='macro', zero_division=0)
recall_macro = recall_score(y_test, y_test_recall_mac, average='macro')
f1_macro = f1_score(y_test, y_test_f1_mac, average='macro')
precision_micro = precision_score(y_test, y_test_prec_mic, average='micro', zero_division=0)
recall_micro = recall_score(y_test, y_test_recall_mic, average='micro')
f1_micro = f1_score(y_test, y_test_f1_mic, average='micro')

print("Test Subtopic Bag of Words Macro Precision:", precision_macro)
print("Test Subtopic Bag of Words Macro Recall:", recall_macro)
print("Test Subtopic Bag of Words Macro F1 Score:", f1_macro)
print("Test Subtopic Bag of Words Micro Precision:", precision_micro)
print("Test Subtopic Bag of Words Micro Recall:", recall_micro)
print("Test Subtopic Bag of Words Micro F1 Score:", f1_micro)

print("\n")

subtopic_bow_kappa = cohen_kappa_score(y_test, y_test_f1_mac)
print("Subtopic Bag of Words: Cohen's kappa:", subtopic_bow_kappa)

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/s

GridSearchCV(cv=5, estimator=LinearSVC(dual='auto'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='f1_macro')

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/s

GridSearchCV(cv=5, estimator=LinearSVC(dual='auto'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='f1_micro')

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in l

...

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/s

GridSearchCV(cv=5, estimator=LinearSVC(dual='auto'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='recall_macro')

/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kijeong/anaconda3/lib/python3.11/site-packages/sklearn/s

GridSearchCV(cv=5, estimator=LinearSVC(dual='auto'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='recall_micro')

Validation Subtopic Bag of Words Macro Precision: 0.48758379542851193
Validation Subtopic Bag of Words Macro Recall: 0.3552007860560492
Validation Subtopic Bag of Words Macro F1 Score: 0.36529220682627284
Validation Subtopic Bag of Words Micro Precision: 0.7699999999999999
Validation Subtopic Bag of Words Micro Recall: 0.7699999999999999
Validation Subtopic Bag of Words Micro F1 Score: 0.7699999999999999


Test Subtopic Bag of Words Macro Precision: 0.5871913580246914
Test Subtopic Bag of Words Macro Recall: 0.37909211029389484
Test Subtopic Bag of Words Macro F1 Score: 0.3958899715305345
Test Subtopic Bag of Words Micro Precision: 0.756
Test Subtopic Bag of Words Micro Recall: 0.756
Test Subtopic Bag of Words Micro F1 Score: 0.756


Subtopic Bag of Words: Cohen's kappa: 0.21203088350273658


# Weblink, '!', '?', Allcap Feature

Next, weblink, '!', '?', and allcap word presence is checked in comments. If weblink is present, Subtopic label is set to "other". Otherwise, it is categorized depending on how many counts the comment has in each of '!', '?', and allcap words there are.

In [32]:
import re

class WeblinkClassifier():
    def __init__(self):
        self.http_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        
    def subtopic_web_predict(self, sentence):
        weblink_count = 0
        exc_count = 0
        cap_count = 0
        q_count = 0
        for word in sentence.lower().split():
            weblink_count = len(re.findall(self.http_pattern, sentence))
            if '!' in word:
                exc_count += 1
            if '?' in word:
                q_count += 1
        for word in sentence.split():
            cap_count = len(re.findall(r'\b[A-Z]+\b', sentence))
            
        subtopic_web_pred = 'other'
        if weblink_count > 0:
            subtopic_web_pred = 'other'
        elif exc_count > cap_count and exc_count > q_count:
            subtopic_web_pred = 'recreation'
        elif cap_count > exc_count and cap_count > q_count:
            subtopic_web_pred = 'mobility'
        elif q_count > exc_count and q_count > cap_count:
            subtopic_web_pred = 'education'
        return subtopic_web_pred

web_class = WeblinkClassifier()
subtopic_web_preds = []
tmp = ''
for feature in X_train:
    tmp = web_class.subtopic_web_predict(feature)
    subtopic_web_preds.append(tmp)

precision_mac = precision_score(y_train, subtopic_web_preds, average='macro', zero_division=0)
recall_mac = recall_score(y_train, subtopic_web_preds, average='macro', zero_division=0)
f1_mac = f1_score(y_train, subtopic_web_preds, average='macro')
precision_min = precision_score(y_train, subtopic_web_preds, average='micro', zero_division=0)
recall_min = recall_score(y_train, subtopic_web_preds, average='micro', zero_division=0)
f1_min = f1_score(y_train, subtopic_web_preds, average='micro')

print("Validation Subtopic Weblink Macro Precision: {:.4f}".format(precision_mac))
print("Validation Subtopic Weblink Macro Recall: {:.4f}".format(recall_mac))
print("Validation Subtopic Weblink Macro F1: {:.4f}".format(f1_mac))
print("Validation Subtopic Weblink Micro Precision: {:.4f}".format(precision_min))
print("Validation Subtopic Weblink Micro Recall: {:.4f}".format(recall_min))
print("Validation Subtopic Weblink Micro F1: {:.4f}".format(f1_min))
print("\n")



subtopic_web_preds = []
tmp = ''
for feature in X_test:
    tmp = web_class.subtopic_web_predict(feature)
    subtopic_web_preds.append(tmp)

precision_mac = precision_score(y_test, subtopic_web_preds, average='macro', zero_division=0)
recall_mac = recall_score(y_test, subtopic_web_preds, average='macro', zero_division=0)
f1_mac = f1_score(y_test, subtopic_web_preds, average='macro')
precision_min = precision_score(y_test, subtopic_web_preds, average='micro', zero_division=0)
recall_min = recall_score(y_test, subtopic_web_preds, average='micro', zero_division=0)
f1_min = f1_score(y_test, subtopic_web_preds, average='micro')
print("Test Subtopic Weblink Macro Precision: {:.4f}".format(precision_mac))
print("Test Subtopic Weblink Macro Recall: {:.4f}".format(recall_mac))
print("Test Subtopic Weblink Macro F1: {:.4f}".format(f1_mac))
print("Test Subtopic Weblink Micro Precision: {:.4f}".format(precision_min))
print("Test Subtopic Weblink Micro Recall: {:.4f}".format(recall_min))
print("Test Subtopic Weblink Micro F1: {:.4f}".format(f1_min))
print("\n")

subtopic_web_kappa = cohen_kappa_score(y_test, subtopic_web_preds)
print("Subtopic Modififed Weblink's Cohen's kappa:", subtopic_web_kappa)

Validation Subtopic Weblink Macro Precision: 0.1917
Validation Subtopic Weblink Macro Recall: 0.2085
Validation Subtopic Weblink Macro F1: 0.1623
Validation Subtopic Weblink Micro Precision: 0.4040
Validation Subtopic Weblink Micro Recall: 0.4040
Validation Subtopic Weblink Micro F1: 0.4040


Test Subtopic Weblink Macro Precision: 0.3094
Test Subtopic Weblink Macro Recall: 0.3073
Test Subtopic Weblink Macro F1: 0.2443
Test Subtopic Weblink Micro Precision: 0.4060
Test Subtopic Weblink Micro Recall: 0.4060
Test Subtopic Weblink Micro F1: 0.4060


Subtopic Modififed Weblink's Cohen's kappa: 0.04437694663955327
